In [8]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [26]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage,BaseOutputParser

In [24]:
chat = ChatOpenAI( )

In [32]:
class CommaOutputParser(BaseOutputParser):

    def parse(self, text: str):
        items = text.strip().split(",")
        return list(map(str.strip,items))

In [36]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a list of max {max_items}. Do NOT reply with anything else."),
    ("human", "{question}"),
])

In [42]:
chain = template | chat | CommaOutputParser()
chain.invoke({"max_items":5, "question":"What are the pokemons?"})

['1. Pikachu\n2. Bulbasaur\n3. Charizard\n4. Squirtle\n5. Jigglypuff']